<font size="4">
    <h1>
        0 - Exploring Video Games through data using Pyspark
    </h1>

<font size="4">
    
Ce notebook accompagne le premier article de la série "Exploring Video Games though data using Pyspark"

Le liens vers les datasets sont présents dans l'article pour exécuter le notebook

C'est parti !
    
<img src = "https://memegenerator.net/img/images/14734203.jpg"
     alt = "memegenerator.net"
     width = 50%
     height = 50%
     >
<font size=1> <div style="text-align:center"> 
    Image via https://memegenerator.net/img/images/14734203.jpg 
</div> </font>

     


<font size="4"> 1. Importer les **packages** nécessaires et configuration du notebook</font>

In [1]:
# Just take all width for viz if you have a wide screen like me.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Use findspark to set Pyspark in sys.path
import findspark
findspark.init()

# import pyspark dependencies
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

import os

<font size="4">  2. Initialisation de la **Spark Session** </font>

In [2]:
spark = SparkSession \
    .builder \
    .master('local[*]') \
    .config("spark.driver.memory", "10g") \
    .appName("steam-analysis-eda") \
    .getOrCreate()

<font size="4">  3. Chargement du fichier Parquet en **Pyspark DataFrame** </font>

In [3]:
# Build dataset path using path to git repo
dataset_path = os.path.join(os.path.dirname(os.path.realpath("")), "data/steam-dataset/")

df = spark.read.parquet('file://' + dataset_path + "steam_analysis.App_ID_Info")

df.printSchema()

root
 |-- appid: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Release_Date: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Required_Age: string (nullable = true)
 |-- Is_Multiplayer: string (nullable = true)



<font size="4"> 4. Extraction des 10 app_id les plus **récentes** </font>

In [4]:
df \
    .sort(F.col('Release_Date').desc()) \
    .select('Title', 'Type', 'Release_Date') \
    .show(10, truncate = False)

+-------------------------------------------------------+---------------+-------------------+
|Title                                                  |Type           |Release_Date       |
+-------------------------------------------------------+---------------+-------------------+
|Title                                                  |Type           |Release_Date       |
|"DLC""The Rainy Port Keelung - Radio Drama""(Only audio| no subtitles)"|8.99               |
|Nefarious                                              |game           |2017-01-01 00:00:00|
|Ara Fell Demo                                          |demo           |2016-12-30 00:00:00|
|Camp Sunshine                                          |game           |2016-10-31 00:00:00|
|Mainlining                                             |game           |2016-10-01 00:00:00|
|Red Comrades 2: For the Great Justice                  |game           |2016-09-22 00:00:00|
|Farabel                                                |gam

<font size="4">
    <li> Deux anomalies ont été détectées à l'aide du sorting. Elles seront traitées plus bas. </li>
</font>

<font size="4">  5. Extraction des 10 app_id les plus **anciennces** </font>

In [19]:
df \
    .sort(F.col('Release_Date')) \
    .select('Title', 'Type', 'Release_Date') \
    .show(10, truncate = False)

+------------------------------------------------+----+-------------------+
|Title                                           |Type|Release_Date       |
+------------------------------------------------+----+-------------------+
|"Magic 2012 Full Deck ""Cloudburst"""           |dlc |1970-01-01 00:00:00|
|The Crew Demo                                   |demo|1970-01-01 00:00:00|
|Swift                                           |game|1970-01-01 00:00:00|
|Sealed Play Deck - Slot 08                      |dlc |1970-01-01 00:00:00|
|"Magic 2012 Full Deck ""Auramancer"""           |dlc |1970-01-01 00:00:00|
|Littlstar VR Cinema                             |game|1970-01-01 00:00:00|
|Candle                                          |game|1970-01-01 00:00:00|
|Pizza Frenzy Deluxe Free Demo                   |demo|1970-01-01 00:00:00|
|Rochard Soundtrack and Artbook                  |dlc |1970-01-01 00:00:00|
|Men of War: Vietnam Special Edition Upgrade Pack|dlc |1970-01-01 00:00:00|
+-----------

<font size="4">  La valeur "1970-01-01 00:00:00" est anormale et devrait être considérée comme une valeur manquante. Pour l'analyse, nous devons filtrer cette valeur.

In [20]:
df \
    .filter(F.col('Release_Date') != "1970-01-01 00:00:00") \
    .sort(F.col('Release_Date')) \
    .select('Title', 'Type', 'Release_Date') \
    .show(10, truncate = False)

+-------------------------+-----+-------------------+
|Title                    |Type |Release_Date       |
+-------------------------+-----+-------------------+
|Mad Max (1979)           |video|1979-04-12 00:00:00|
|Carmageddon Max Pack     |game |1997-06-30 00:00:00|
|Half-Life                |game |1998-11-08 00:00:00|
|Team Fortress Classic    |game |1999-04-01 00:00:00|
|Half-Life: Opposing Force|game |1999-11-01 00:00:00|
|Ricochet                 |game |2000-11-01 00:00:00|
|Counter-Strike           |game |2000-11-01 00:00:00|
|Gothic 1                 |game |2001-03-15 00:00:00|
|Half-Life: Blue Shift    |game |2001-06-01 00:00:00|
|Deathmatch Classic       |game |2001-06-01 00:00:00|
+-------------------------+-----+-------------------+
only showing top 10 rows



<font size="4"> Nous pouvons voir que l'appid la plus ancienne est une vidéo. Nous voulons considérer les jeux uniquement, nous allons donc également filtrer par type.

In [21]:
df \
    .filter(F.col('Release_Date') != "1970-01-01 00:00:00") \
    .filter(F.col('Type') == "game") \
    .sort(F.col('Release_Date')) \
    .select('Title', 'Type', 'Release_Date') \
    .show(10, truncate = False)

+-------------------------+----+-------------------+
|Title                    |Type|Release_Date       |
+-------------------------+----+-------------------+
|Carmageddon Max Pack     |game|1997-06-30 00:00:00|
|Half-Life                |game|1998-11-08 00:00:00|
|Team Fortress Classic    |game|1999-04-01 00:00:00|
|Half-Life: Opposing Force|game|1999-11-01 00:00:00|
|Counter-Strike           |game|2000-11-01 00:00:00|
|Ricochet                 |game|2000-11-01 00:00:00|
|Gothic 1                 |game|2001-03-15 00:00:00|
|Half-Life: Blue Shift    |game|2001-06-01 00:00:00|
|Deathmatch Classic       |game|2001-06-01 00:00:00|
|Geneforge 1              |game|2001-12-01 00:00:00|
+-------------------------+----+-------------------+
only showing top 10 rows



<font size="4">  Nous apliquons également le filtre sur le type aux 10 appid les plus récentes.

In [22]:
df \
    .filter(F.col('Type') == "game") \
    .sort(F.col('Release_Date').desc()) \
    .select('Title', 'Type', 'Release_Date') \
    .show(10, truncate = False)

+--------------------------------------------+----+-------------------+
|Title                                       |Type|Release_Date       |
+--------------------------------------------+----+-------------------+
|Nefarious                                   |game|2017-01-01 00:00:00|
|Camp Sunshine                               |game|2016-10-31 00:00:00|
|Mainlining                                  |game|2016-10-01 00:00:00|
|Red Comrades 2: For the Great Justice       |game|2016-09-22 00:00:00|
|Farabel                                     |game|2016-09-15 00:00:00|
|Avadon 3: The Warborn                       |game|2016-09-14 00:00:00|
|Seinarukana -The Spirit of Eternity Sword 2-|game|2016-09-01 00:00:00|
|End Of The Mine                             |game|2016-09-01 00:00:00|
|RIVE                                        |game|2016-09-01 00:00:00|
|Empathy                                     |game|2016-09-01 00:00:00|
+--------------------------------------------+----+-------------

<font size="4">
6. Extraction du le nombre de jeux released par année.

- Nous devons extraire l'année avant d'exécuter le groupBy (granularité au jour dans les données)

In [38]:
df \
    .filter(F.col('Release_Date') != "1970-01-01 00:00:00") \
    .withColumn('release_year', F.year('Release_Date')) \
    .groupBy(F.col('release_year')) \
    .agg(F.size(F.collect_list('Title')).alias('N_records')) \
    .dropna() \
    .sort(F.col('release_year')) \
    .show(50)

+------------+---------+
|release_year|N_records|
+------------+---------+
|        1979|        1|
|        1997|        1|
|        1998|        1|
|        1999|        2|
|        2000|        2|
|        2001|        4|
|        2003|        3|
|        2004|        8|
|        2005|        8|
|        2006|       80|
|        2007|      148|
|        2008|      246|
|        2009|      491|
|        2010|      479|
|        2011|      609|
|        2012|     1210|
|        2013|     1546|
|        2014|     3510|
|        2015|     5995|
|        2016|     3012|
|        2017|        1|
+------------+---------+



In [10]:
# Convert results to pandas DataFrame for Plotly compatibility
pd_df = df \
    .filter(F.col('Release_Date') != "1970-01-01 00:00:00") \
    .withColumn('release_year', F.year('Release_Date')) \
    .groupBy(F.col('release_year')) \
    .agg(F.size(F.collect_list('Title')).alias('N_records')) \
    .dropna() \
    .sort(F.col('release_year')) \
    .toPandas()

pd_df

,release_year,N_records
0,1979,1
1,1997,1
2,1998,1
3,1999,2
4,2000,2
5,2001,4
6,2003,3
7,2004,8
8,2005,8
9,2006,80


In [8]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(
    go.Bar(
        x = pd_df.release_year,
        y = pd_df.N_records,
        name = "Number of released games"
    )
)

fig.update_layout(
    title = 'Sum of released games by year on Steam platform',
    template = 'plotly_dark+presentation'
)

fig.show()

<font size="4"> 
    Récupérer toutes les release_year <b>distinctes</b>
</font>

In [82]:
df \
    .withColumn('release_year', F.year('Release_Date')) \
    .select('release_year') \
    .distinct() \
    .sort(F.col('release_year')) \
    .show(50) # Choose to show top 50 rows, greatly larger than expected year span

+------------+
|release_year|
+------------+
|        null|
|        1970|
|        1979|
|        1997|
|        1998|
|        1999|
|        2000|
|        2001|
|        2003|
|        2004|
|        2005|
|        2006|
|        2007|
|        2008|
|        2009|
|        2010|
|        2011|
|        2012|
|        2013|
|        2014|
|        2015|
|        2016|
|        2017|
+------------+



<font size="4"> 

On peut voir qu'il y a deux valeurs étranges : null et "1970"

La valeur "null" est produite par F.year en cas de mauvais formattage de la string.

La valeur "1970" ne peut bien entendu pas être juste, et correspond à timestamp = 0. On peut donc considérer qu'il s'agit d'une valeur manquante.

In [79]:
df \
    .withColumn('release_year', F.year('Release_Date')) \
    .filter(F.col('release_year').isNull()) \
    .count()

2

In [78]:
df \
    .withColumn('release_year', F.year('Release_Date')) \
    .filter(F.col('release_year').isNull()) \
    .show()

+------+--------------------+---------------+-----+------------+-------------------+------------+--------------+------------+
| appid|               Title|           Type|Price|Release_Date|             Rating|Required_Age|Is_Multiplayer|release_year|
+------+--------------------+---------------+-----+------------+-------------------+------------+--------------+------------+
| appid|               Title|           Type|Price|Release_Date|             Rating|Required_Age|Is_Multiplayer|        null|
|361060|"DLC""The Rainy P...| no subtitles)"|  dlc|        8.99|2015-04-06 00:00:00|          -1|             0|        null|
+------+--------------------+---------------+-----+------------+-------------------+------------+--------------+------------+



<font size="4"> 

#### Workflow complet :

- Filtrer les résultats datés de 1970
- Filtrer les anomalies (headers + records mal formattés)
- Visualiser les résultats propres (optional)

In [9]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import findspark
findspark.init()

from pyspark.sql import SparkSession
import pyspark.sql.functions as F

import os

import plotly.graph_objects as go

# Get Spark session
spark = SparkSession \
    .builder \
    .master('local[*]') \
    .config("spark.driver.memory", "10g") \
    .appName("steam-analysis-eda") \
    .getOrCreate()

dataset_path = "/home/neadex/steam-analysis/data/steam-dataset/"
df = spark.read.parquet('file://' + dataset_path + "steam_analysis.App_ID_Info")

# Get dataframe with all filters
# Filter for 1970 record change to check against int time instead of str type
pd_df = df \
    .withColumn('release_year', F.year('Release_Date')) \
    .filter(F.col('release_year').isNull() == False) \
    .filter(F.col('release_year') > 1970) \
    .groupBy(F.col('release_year')) \
    .agg(F.size(F.collect_list('Title')).alias('N_records')) \
    .dropna() \
    .sort(F.col('release_year')) \
    .toPandas()

# Create plot using Plotly
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x = pd_df.release_year,
        y = pd_df.N_records,
        name = "Number of released games"
    )
)
fig.update_layout(
    title = 'Sum of released games by year on Steam platform',
    template = 'plotly_dark+presentation'
)
fig.show()

<font size="4"> 
    <h1>Conclusions</h1>
</font>

<font size="4"> 

- L'article de recherche accompagnant le dataset a été publié en Novembre 2016 (https://dl.acm.org/doi/10.1145/2987443.2987489). 
Les données de 2016 ne peuvent donc pas être complètes (d'autant plus en considérant le temps de rédaction et de validation de l'article !)
<br><b>L'analyse ignorera donc les données post-2015.</b></br>


- On peut également voir que le volume de données commence à prendre de l'importance seulement à partir de 2006, bien que 2005 et 2004 ne soient pas nulles (8 releases pour 2004 et 2005). <br>
L'explication ici n'est pas technique mais historique. En effet, en vérifiant l'historique de Steam sur Wikipedia (https://fr.wikipedia.org/wiki/Steam), on peut constater que la plateforme ne s'est ouverte qu'aux jeux tiers qu'à partir de 2005. <b>L'analyse ignorera les données pré-2005.</b>
</br>


- L'analyse sera donc <b>limitée aux années entre 2005 et 2015 inclus</b>. Il sera donc possible par la suite d'utiliser les données de 2016 à 2020 pour vérifier les prédictions qui seront émises plus tard.


- La dernière cellule peut-être soumise en tant qu'**application** Spark pour automatiser le processus.

<img 
     src="https://47billion.com/wp-content/uploads/2020/02/game-analytics.png" 
     alt="https://47billion.com/blog/how-gaming-industry-drives-customer-engagement/"
     width=60% 
     height=60
     >
     
<font size=1> <div style="text-align:center"> 
    Image via https://47billion.com/blog/how-gaming-industry-drives-customer-engagement/
</div>